In [ ]:
!sudo pip install pandas xlrd

In [ ]:
def log(string, alsoPrint):
    with open(logFilePath, "w") as logFileBuffer:
        logFileBuffer.write(f"[{datetime.now()}]:  {string}")
        
    if alsoPrint:
        print(string)

In [ ]:
## This file will test the capability to read in attendance sheets
# We use the pandas library to read in data from excel files
import pandas as pd

def parseSheet(sheetName):
    
    debug = True
    
    # The test file we will be working on
    #sheetName = "Career Studio Takeover.xlsx"

    try:
        # Load in the excel file using pandas
        file = pd.read_excel(sheetName)
    except:
        print(f'File {sheetName} not found!')
        return
    
    #print(file.columns)

    # We can't expect that everyone will format the name exactly as it should be,
    # so we want to identify what each header is actually called
    # Below we have a list of words that should show up in one way or another for each category
    keywords = {'Student ID':["id", 'student', 'ncsu'],
                'Event Name':['event', 'program', 'name'],
                'Student Name':['name', 'first'],
                'Village Points':['village', 'points', 'vp'],
                'Hosts':['host', 'fellow', 'leader'],
                'Date':['date', 'day', 'event', 'program']}
    actualKeywords = {}

    # These are characters that may show up that we don't want to influence the identification process
    arbitraryChars = [':', '(', ')', '[', ']', '{', '}']

    for header in file.columns:
        # Make it lowercase
        currHeader = header.lower()

        # We want to ignore columns that don't have headers
        if 'Unnamed' in header:
            continue

        # Remove semicolons
        for c in arbitraryChars:
            currHeader = currHeader.replace(c, '')

        #print(currHeader)

        qualifierMatches = {}
        i = 0
        for keyword, qualifiers in keywords.items():
            qualifierMatches[keyword] = 0
            for q in qualifiers:
                if q in currHeader:
                    qualifierMatches[keyword] = qualifierMatches[keyword] + 1

            # This is kinda an obscure way of weighting things, but it seems to work for now
            qualifierMatches[keyword] = float(qualifierMatches[keyword]) / (float(len(currHeader.split())) * float(len(keywords[keyword])))
            i = i + 1

        # Now we want to find which index of qualifierMatches has the highest value, which will become the actualKeyword
        key = max(qualifierMatches, key=qualifierMatches.get)
        #print(key)
        actualKeywords[key] = header

        #print(qualifierMatches)

    #print(actualKeywords)

    # We now know where all of our information is, so we should start grabbing it
    possibleVillagePoints = file[actualKeywords['Village Points']].values.tolist()
    # This removes all instance of 'nan' ie. empty cells
    possibleVillagePoints = possibleVillagePoints[possibleVillagePoints == possibleVillagePoints]
    #possibleVillagePoints = possibleVillagePoints.remove(None)
    print(possibleVillagePoints)
    
    possibleDates = file[actualKeywords['Date']].values.tolist()
    print(possibleDates)

    while None in possibleDates:
        possibleDates.remove(None)
    print(possibleDates)
    
    
    possibleHosts = file[actualKeywords['Hosts']].values.tolist()
    print(possibleHosts)
    # This removes all instance of 'nan' ie. empty cells
    possibleHosts = possibleHosts[not possibleHosts == possibleHosts]
    print(possibleHosts)

    possibleNames = file[actualKeywords['Event Name']].values.tolist()
    print(possibleNames)
    # This removes all instance of 'nan' ie. empty cells
    possibleNames = possibleNames[not possibleNames == possibleNames]
    print(possibleNames)

    villagePoints = file[actualKeywords['Village Points']][0]
    eventDate = possibleDates[0]
    hosts = file[actualKeywords['Hosts']][0]
    eventName = file[actualKeywords['Event Name']][0]

    
    print([villagePoints, eventDate, hosts, eventName])

    students = file[actualKeywords['Student ID']]
    print(students)
    
    # Now that we have all of the information about the event, we should return all of the students who atended and the information about the event
    return [villagePoints, eventDate, hosts, eventName, students]


In [ ]:
parseSheet('Career Studio Takeover.xlsx')

In [ ]:
import math

In [ ]:
? list.remove